In [26]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

In [ ]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

k = 1 status = f


In [29]:
origin = 'n1'
dests = ['n22', 'n66']

def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = 0.5, # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st == 's':
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', 1) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

row_names = list(edges.keys()) + ['sys']
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


In [41]:
result = tsum.run_rule_extraction(
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    output_dir="tsum_res",
    surv_json_name="rules_surv.json",
    fail_json_name="rules_fail.json",
    unk_prob_thres = 1e-6
) 

---
Round: 1, Unk. prob.: 1.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from survival rules 👍
No. of existing rules removed:  0
New rule added. System state: s, System value: 0.517714415076909. Total samples: 1024.
New rule (No. of conditions: 4): {'e0003': ('>=', 1), 'e0019': ('>=', 1), 'e0024': ('>=', 1), 'e0027': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [0.518]
Failure sample found from failure rules 👍
No. of existing rules removed:  0
New rule added. System state: f, System value: None. Total samples: 1024.
New rule (No. of conditions: 3): {'e0001': ('<=', 0), 'e0002': ('<=', 0), 'e0003': ('<=', 0), 'sys': ('<=', 0)}
Unique system values: [0.518, None]
---
Round: 2, Unk. prob.: 1.000e+00
No. of non-dominant rules: 2, Survival rules: 1, Failure rules: 1
Survival sample found from survival rules 👍
No. of existing rules removed:  0
New rule added. System state: s, System value: 0.521755546281711. Total samples: 1024.
New rule (